## 규칙
- 각 이름은 띄어쓰기 없이 작성하기. 예를 들어, "영업 시간"이 아니라 "영업시간"으로

In [1]:
from tqdm import tqdm
from langchain_community.vectorstores import FAISS
import pickle
import pandas as pd
import numpy as np
from langchain.schema import Document
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
class CFG:
    output_path = "/home/user09/beaver/data/db"
    data_path = ""
    save_path = ""
    embedding_model= "nlpai-lab/KoE5" # "BAAI/bge-m3"
    retriever_k=5
    retriever_bert_weight=0.7
    version='_SQL_general_ju_1'
    # info_type='STORE_INFO'   # STORE_INFO, TIME_INFO, MENU_INFO
    store_num = 105200  # 103017(미스), 104049(홍콩), 104562(새마을), 104570(롤링), 105200(우아)
    fill_nan = "None"  # 없는 정보의 경우에는 "정보없음"보다는 "None"이 나은 듯
    # basic_info = ["주차장", "씨씨티비", "영업시간", "예약가능여부", "전화번호"]
    seed=42 


CFG.data_path = f"/home/user09/beaver/data/shared_files/dataset/dataset{CFG.version}.xlsx"
print(CFG.data_path)
CFG.save_path = f"/home/user09/beaver/data/shared_files/dataset/dataset{CFG.version}_{CFG.store_num}_preprocessed.xlsx"
print(CFG.save_path)

/home/user09/beaver/data/shared_files/dataset/dataset_SQL_general_ju_1.xlsx
/home/user09/beaver/data/shared_files/dataset/dataset_SQL_general_ju_1_105200_preprocessed.xlsx


### STORE_INFO

In [3]:
################ 전처리 ################
from langchain.schema import Document

# ['STORE_NO', 'STORE_NM', 'BRAND_NO', 'BRAND_NM', 'ROAD_NM', 'BSPLC_TELNO', 'X', 'Y', 'T_CNT', 'P_CNT']
columns_to_use = ['STORE_NO', 'STORE_NM', 'ROAD_NM', 'T_CNT']
df_store = pd.read_excel(CFG.data_path, sheet_name='STORE_INFO', usecols=columns_to_use)

df_store = df_store[df_store['STORE_NO']==CFG.store_num]  # 해당 매장 정보만 가져옴
store = df_store['STORE_NM'].unique()[0]
print(store)

df_store.drop(columns=['STORE_NO'], inplace=True)

df_store_col = df_store.columns

# 결과 출력
print(df_store)
print(df_store_col)

우아하계 완정역점
    STORE_NM           ROAD_NM  T_CNT
4  우아하계 완정역점  인천 서구 완정로10번길 16     18
Index(['STORE_NM', 'ROAD_NM', 'T_CNT'], dtype='object')


In [4]:
############## 컬럼명 변경 ##############
# ['STORE_NO', 'STORE_NM', 'BRAND_NO', 'BRAND_NM', 'ROAD_NM', 'BSPLC_TELNO', 'X', 'Y', 'T_CNT', 'P_CNT']
store_column_mapping = {
    'STORE_NM': 'STORE_NM',
    'ROAD_NM': 'ROAD_NM_ADDR',
    'T_CNT': 'TABLE_NM',  # !!!!!!!!!!!!!!!!!
}

for col in df_store_col:
    df_store = df_store.rename(columns={col: store_column_mapping[col]})
    
print(df_store)

    STORE_NM      ROAD_NM_ADDR  TABLE_NM
4  우아하계 완정역점  인천 서구 완정로10번길 16        18


### PATMNT_INFO

In [5]:
import pandas as pd

# 엑셀 파일에서 필요한 컬럼만 가져오기
columns_to_use = ['STORE_NO', 'PAYMNT_MN', 'EASY_PAYMNT_TYPE_NM']
df_pay = pd.read_excel(CFG.data_path, sheet_name='PAYMNT_INFO', usecols=columns_to_use)

# 매장 번호 필터링 (103017로 변경)
df_pay = df_pay[df_pay['STORE_NO'] == CFG.store_num]  # 매장 번호에 맞는 데이터만 가져옴
df_pay.drop(columns=['STORE_NO'], inplace=True)

# EASY_PAYMNT_TYPE_NM의 첫 번째 값 처리 (결측값 확인 후 진행)
# if not df_pay['EASY_PAYMNT_TYPE_NM'].dropna().empty:
simple_pay = str(df_pay['EASY_PAYMNT_TYPE_NM'].dropna().iloc[0])  # 첫 번째 비어있지 않은 값 가져오기


# PAYMNT_MN 컬럼을 문자열로 변환
df_pay['PAYMNT_MN'] = df_pay['PAYMNT_MN'].astype(str)

# 간편결제 문자열을 조건에 맞게 결합
# if simple_pay:  # simple_pay가 None이 아닐 때만 수행
#     df_pay.loc[df_pay['PAYMNT_MN'].str.strip() == '간편결제', 'PAYMNT_MN'] += "(" + simple_pay + ")"
# df_pay.drop(columns=['EASY_PAYMNT_TYPE_NM'], inplace=True)

df_pay_col = df_pay.columns

# 결과 출력
print(df_pay)
print(df_pay_col)

   PAYMNT_MN                                EASY_PAYMNT_TYPE_NM
28      신용카드                                                NaN
29        현금                                                NaN
30    모바일상품권                                                NaN
31       포인트                                                NaN
32      매장쿠폰                                                NaN
33       선불권                                                NaN
34      간편결제  네이버페이, 비플제로페이, 삼성페이, 알리페이, 앱결제, 앱카드, 위챗페이, 카카오...
Index(['PAYMNT_MN', 'EASY_PAYMNT_TYPE_NM'], dtype='object')


In [6]:
############## 컬럼명 변경 ##############
# ['STORE_NO', 'PAYMNT_MN', 'EASY_PAYMNT_TYPE_NM']
pay_column_mapping = {
    'PAYMNT_MN': 'PAYMNT_MN_CD',
    'EASY_PAYMNT_TYPE_NM': 'EASY_PAYMNT_TYPE_CD',
}

for col in df_pay_col:
    df_pay = df_pay.rename(columns={col: pay_column_mapping[col]})

print(df_pay)

   PAYMNT_MN_CD                                EASY_PAYMNT_TYPE_CD
28         신용카드                                                NaN
29           현금                                                NaN
30       모바일상품권                                                NaN
31          포인트                                                NaN
32         매장쿠폰                                                NaN
33          선불권                                                NaN
34         간편결제  네이버페이, 비플제로페이, 삼성페이, 알리페이, 앱결제, 앱카드, 위챗페이, 카카오...


In [7]:
# 간편결제 행 찾기
easy_pay_row = df_pay[df_pay['PAYMNT_MN_CD'] == 'EASY_PAYMNT_TYPE_CD']
if not easy_pay_row.empty:
    # 간편결제수단 값 추출
    easy_pay_method = easy_pay_row['EASY_PAYMNT_TYPE_CD'].iloc[0]
else:
    easy_pay_method = np.nan
    
payment_methods = df_pay['PAYMNT_MN_CD'].tolist()
payment_methods_str = ", ".join(payment_methods)

df_pay = pd.DataFrame({
    'PAYMNT_MN_CD': [payment_methods_str],
    'EASY_PAYMNT_TYPE_CD': [easy_pay_method if pd.notna(easy_pay_method) else '없음']
})

print(df_pay)

                             PAYMNT_MN_CD EASY_PAYMNT_TYPE_CD
0  신용카드, 현금, 모바일상품권, 포인트, 매장쿠폰, 선불권, 간편결제                  없음


In [9]:
df_store.reset_index(drop=True, inplace=True)
df_pay.reset_index(drop=True, inplace=True)
df_ = pd.concat([df_store, df_pay], axis=1)
print(df_)

    STORE_NM      ROAD_NM_ADDR  TABLE_NM  \
0  우아하계 완정역점  인천 서구 완정로10번길 16        18   

                             PAYMNT_MN_CD EASY_PAYMNT_TYPE_CD  
0  신용카드, 현금, 모바일상품권, 포인트, 매장쿠폰, 선불권, 간편결제                  없음  


### MENU_INFO

In [56]:
################ 전처리 ################
# ['STORE_NO', 'BRAND_NO', 'CATEGORY_NO', 'CATEGORY_NM', 'STD_CATEGORY_NM', 'PROD_NO', 'PROD_HIST_NO', 'PROD_NM', 'STD_PROD_NM', 'SET_PROD_YN', 'ORGIN_INDI', 'INDI_TYPE_NM', 'PROD_BASE_PRICE', 'QTY', 'OPTION_GROUP_NO', 'OPTION_GROUP_NM', 'MIN_SEL_CNT' ,'MAX_SEL_CNT', 'ESS_YN', 'OPTION_PROD_NO', 'OPTION_PROD_NM', 'OPTION_PROD_PRICE']
columns_to_use = ['STORE_NO', 'STD_CATEGORY_NM', 'STD_PROD_NM', 'QTY']
df_menu = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset{CFG.version}.xlsx', sheet_name='MENU_INFO', usecols=columns_to_use)
df_menu = df_menu[df_menu['STORE_NO']==CFG.store_num]
df_menu.drop(columns=['STORE_NO'], inplace=True)
df_menu['STD_CATEGORY_NM'] = df_menu['STD_CATEGORY_NM'].mask(df_menu['STD_CATEGORY_NM'].duplicated(), '')

print(df_menu[:3])
print(df_menu)

    STD_CATEGORY_NM STD_PROD_NM  QTY
498            메인메뉴   우아하계 한판세트  321
499                  숯불닭갈비갈비양념   297
500                  숯불닭갈비소금구이   296
    STD_CATEGORY_NM STD_PROD_NM  QTY
498            메인메뉴   우아하계 한판세트  321
499                  숯불닭갈비갈비양념   297
500                  숯불닭갈비소금구이   296
501                      닭목살구이    84
502                    닭목살구이양념    37
503                     닭안창살구이    30
504                   닭안창살구이양념    34
505                    닭염통양념구이    28
506                     무뼈닭발구이    51
507                      한판세트 B   30
508           사이드메뉴       능이닭개장   73
509                        쑥갓된장   40
510                       잔치닭국수  186
511                       비빔닭국수  161
512                         주먹밥  138
513                         공기밥  277
514                       또띠아추가   88
515                      꽈리고추추가   18
516                       떡사리추가   14
517              음료        제로콜라   78
518                        환타파인   71
519                       스프라이트   67
5

In [57]:
df__ = pd.concat([df_, df_menu], ignore_index=True)
print(df__)

     STORE_NM      ROAD_NM_ADDR  TABLE_NM  \
0   우아하계 완정역점  인천 서구 완정로10번길 16      18.0   
1         NaN               NaN       NaN   
2         NaN               NaN       NaN   
3         NaN               NaN       NaN   
4         NaN               NaN       NaN   
5         NaN               NaN       NaN   
6         NaN               NaN       NaN   
7         NaN               NaN       NaN   
8         NaN               NaN       NaN   
9         NaN               NaN       NaN   
10        NaN               NaN       NaN   
11        NaN               NaN       NaN   
12        NaN               NaN       NaN   
13        NaN               NaN       NaN   
14        NaN               NaN       NaN   
15        NaN               NaN       NaN   
16        NaN               NaN       NaN   
17        NaN               NaN       NaN   
18        NaN               NaN       NaN   
19        NaN               NaN       NaN   
20        NaN               NaN       NaN   
21        

### TIME_INFO

In [58]:
# 전처리
columns_to_use = ['STORE_NO', 'SALE_BGN_TM', 'SALE_END_TM',]
df_time = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset{CFG.version}.xlsx', sheet_name='TIME_INFO', usecols=columns_to_use)
df_time = df_time[df_time['STORE_NO']==105200]

def convert_time_format(hhmmss):
    if pd.isna(hhmmss) or hhmmss in [None, 'None']:
        return "없음"
    hhmmss = str(int(hhmmss)).zfill(6)
    h = hhmmss[:2]
    m = hhmmss[2:4]
    if m == '00':
        return h + '시'
    else:
        return h + '시' + m + '분'

def handle_missing_values(df, columns):
    for col in columns:
        if df[col].isna().all():
            # 해당 매장의 해당 컬럼의 모든 값이 없으면, 정보가 없는 것으로 간주하여 None값으로 채움
            df[col] = CFG.fill_nan
        else:
            # 일부라도 값이 있으면 해당 요일에 브레이크 타임이 없는 경우 '없음'으로 채움
            df[col] = df[col].fillna("없음")
    return df

# 시간 변환 함수 적용
df_time['SALE_BGN_TM'] = df_time['SALE_BGN_TM'].apply(convert_time_format)
df_time['SALE_END_TM'] = df_time['SALE_END_TM'].apply(convert_time_format)

df_time.drop(columns=['STORE_NO'], inplace=True)

print(df_time)


   SALE_BGN_TM SALE_END_TM
10         10시         22시


In [61]:
df___ = pd.concat([df__, df_time], ignore_index=True)
num_rows = df___.shape[0]
# 특정 열만 값 채우고 나머지는 빈 값으로 채우기
df___['BADGE'] = ['점심 특선 메뉴'] + [np.nan] * (num_rows - 1)
df___['SPECIAL_PROD'] = ['능이 닭개장'] + [np.nan] * (num_rows - 1)
df___['ORDER_TYPE'] = ['포장', '배달'] + [np.nan] * (num_rows - 2)
df___['HOLIDAY_TYPE_CD'] = ['일요일'] + [np.nan] * (num_rows - 1)
df___['DLVR_ZONE_RADS'] = ['완정동'] + [np.nan] * (num_rows - 1)
df___['DLVR_TIP_AMT_MIN'] = ['1000원'] + [np.nan] * (num_rows - 1)
df___['DLVR_TIP_AMT_MAX'] = ['3000원'] + [np.nan] * (num_rows - 1)
df___['PACK_YN'] = ['제공하고'] + [np.nan] * (num_rows - 1)
df___['ROOM_SEAT'] = ['룸 좌석'] + [np.nan] * (num_rows - 1)
df___['OUTDOOR_SEAT'] = ['야외 좌석'] + [np.nan] * (num_rows - 1)
df___['ROOM_TABLE_ZONE_NM'] = ['15개'] + [np.nan] * (num_rows - 1)
df___['OUTDOOR_TABLE_ZONE_NM'] = ['5개'] + [np.nan] * (num_rows - 1)
df___['USE_POSBL_BASE_POINT'] = ['1000'] + [np.nan] * (num_rows - 1)
df___['STAMP_TMS'] = ['10'] + [np.nan] * (num_rows - 1)
df___['COUPON_PACK_NM'] = ['할인'] + [np.nan] * (num_rows - 1)
df___['EVENT_NM'] = ['콜라보'] + [np.nan] * (num_rows - 1)
df___['EVENT_BNEF_CND_CD'] = ['두 달'] + [np.nan] * (num_rows - 1)
print(df___)

     STORE_NM      ROAD_NM_ADDR  TABLE_NM  \
0   우아하계 완정역점  인천 서구 완정로10번길 16      18.0   
1         NaN               NaN       NaN   
2         NaN               NaN       NaN   
3         NaN               NaN       NaN   
4         NaN               NaN       NaN   
5         NaN               NaN       NaN   
6         NaN               NaN       NaN   
7         NaN               NaN       NaN   
8         NaN               NaN       NaN   
9         NaN               NaN       NaN   
10        NaN               NaN       NaN   
11        NaN               NaN       NaN   
12        NaN               NaN       NaN   
13        NaN               NaN       NaN   
14        NaN               NaN       NaN   
15        NaN               NaN       NaN   
16        NaN               NaN       NaN   
17        NaN               NaN       NaN   
18        NaN               NaN       NaN   
19        NaN               NaN       NaN   
20        NaN               NaN       NaN   
21        

### 휴무일

In [140]:
# columns_to_use = ['STORE_NO', 'HOLIDAY_TYPE_CD']
# df_holiday = pd.read_excel(f'/home/user09/beaver/data/shared_files/dataset/dataset_v{CFG.version}.xlsx', sheet_name='HOLIDAY_INFO', usecols=columns_to_use)
# df_holiday = df_holiday[df_holiday['STORE_NO']==CFG.store_num]   # 해당 매장 정보만 가져옴
# df_holiday.fillna(CFG.fill_nan, inplace=True)
# df_holiday.drop(columns=['STORE_NO'], inplace=True)
# df_holiday_col = df_holiday.columns
# print(df_holiday)

In [141]:
# ############## 컬럼명 변경 ##############
# day_column_mapping = {
#     # 'PROD_NO': '상품 번호',
#     'HOLIDAY_TYPE_CD': '휴무일',
# }

# for col in df_holiday_col:
#     df_holiday = df_holiday.rename(columns={col: day_column_mapping[col]})
    
# print(df_holiday[:3])

In [142]:
# ############### 한 컬럼으로 통합 ###############
# def create_contents_rowwise(df):
#     contents_data = []
#     for _, row in df.iterrows():
#         row_content = ", ".join([f"'{col}': {row[col]}" for col in df.columns])  # 각 행의 컬럼명:값을 ", "로 결합하여 하나의 문자열 생성
#         contents_data.append(row_content)
    
#     new_df = pd.DataFrame({'contents': contents_data})  # contents 열을 포함하는 새로운 데이터프레임 생성
#     return new_df

# # 함수 호출
# result_df_holiday = create_contents_rowwise(df_holiday)
# print(result_df_holiday[:3])

### 4개 통합 & 피클파일

In [143]:
df = pd.concat([result_df_store, result_df_pay, result_df_menu, result_df_time], ignore_index=True)

In [62]:
print(df___)

     STORE_NM      ROAD_NM_ADDR  TABLE_NM  \
0   우아하계 완정역점  인천 서구 완정로10번길 16      18.0   
1         NaN               NaN       NaN   
2         NaN               NaN       NaN   
3         NaN               NaN       NaN   
4         NaN               NaN       NaN   
5         NaN               NaN       NaN   
6         NaN               NaN       NaN   
7         NaN               NaN       NaN   
8         NaN               NaN       NaN   
9         NaN               NaN       NaN   
10        NaN               NaN       NaN   
11        NaN               NaN       NaN   
12        NaN               NaN       NaN   
13        NaN               NaN       NaN   
14        NaN               NaN       NaN   
15        NaN               NaN       NaN   
16        NaN               NaN       NaN   
17        NaN               NaN       NaN   
18        NaN               NaN       NaN   
19        NaN               NaN       NaN   
20        NaN               NaN       NaN   
21        

### 기본 정보 지정

In [145]:
# ":"를 기준으로 분리하여 앞의 키워드를 추출한 후, 존재 여부 확인
# existing_info = [content.split(":")[0].strip() for content in df["contents"]]
# missing_info = [info for info in CFG.basic_info if info not in existing_info]
# for info in missing_info:
#     df = pd.concat([df, pd.DataFrame({"contents": [f"{info}: 정보없음"]})], ignore_index=True)

In [63]:
# 결과 저장 및 출력
df___.to_excel(CFG.save_path, index=False)   # 전처리한 엑셀 파일 저장
print(df___)

     STORE_NM      ROAD_NM_ADDR  TABLE_NM  \
0   우아하계 완정역점  인천 서구 완정로10번길 16      18.0   
1         NaN               NaN       NaN   
2         NaN               NaN       NaN   
3         NaN               NaN       NaN   
4         NaN               NaN       NaN   
5         NaN               NaN       NaN   
6         NaN               NaN       NaN   
7         NaN               NaN       NaN   
8         NaN               NaN       NaN   
9         NaN               NaN       NaN   
10        NaN               NaN       NaN   
11        NaN               NaN       NaN   
12        NaN               NaN       NaN   
13        NaN               NaN       NaN   
14        NaN               NaN       NaN   
15        NaN               NaN       NaN   
16        NaN               NaN       NaN   
17        NaN               NaN       NaN   
18        NaN               NaN       NaN   
19        NaN               NaN       NaN   
20        NaN               NaN       NaN   
21        

In [147]:
#### 엑셀파일 DB화(pickle파일로 변환) ##### Extracting information from the "Details" column to create documents
docs = []
for _, row in df.iterrows():
    details = row['contents']
    # metadata = {}
    
    # # Parse key-value pairs in the "Details" column
    # for detail in details.split('\n'):
    #     if ':' in detail:
    #         key, value = detail.split(':', 1)
            # metadata[key.strip()] = value.strip()
    
    docs.append(Document(page_content=details))#, metadata=metadata))

# Embeddings and vector store setup
encode_kwargs = {'normalize_embeddings': True}
model_kwargs = {'device': 'cpu'}

hf = HuggingFaceEmbeddings(
    model_name=CFG.embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

db = FAISS.from_documents(docs, hf)

# Save the FAISS vector store and documents as pickle
db.save_local(f"{CFG.output_path}/{CFG.store_num}_faiss{CFG.version}")

with open(f"{CFG.output_path}/{CFG.store_num}_docs{CFG.version}.pkl", "wb") as f:
    pickle.dump(docs, f)

# Load the FAISS vector store
db = FAISS.load_local(f"/home/user09/beaver/data/db/{CFG.store_num}_faiss{CFG.version}", hf, allow_dangerous_deserialization=True)

# Create retrievers
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": CFG.retriever_k})
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = CFG.retriever_k

ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever, bm25_retriever],
    weights=[CFG.retriever_bert_weight, 1 - CFG.retriever_bert_weight]
)
